In [ ]:
import os
import io
import getpass
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pathlib import Path
from reportlab.lib.units import inch
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, PageBreak, Image

# 獲取當前日期並格式化成 'YYYYMM' 的形式 #202407
currentmonth_date = datetime.datetime.now().strftime('%Y%m')
# 獲取當前月份 #7
current_month = datetime.datetime.now().month
# 取得當天的日期 
today = datetime.datetime.now().strftime('%Y%m%d')
# 報告月份與位置
report_file_name = f"CSPM_{currentmonth_date}_Report"
ecv_recommendations = 'ECV_Azure_Recommendations.csv'
# 匯出檔案路徑
username = getpass.getuser() #leo
local_path = Path(rf"/home/{username}/app")
recommendations_path = Path(f'{local_path}/Recommendations/{ecv_recommendations}')
cspm_raw_data_path = Path(rf'{local_path}/cspm_raw_data')
report_path = Path(f'{local_path}/report')

print(f"Debug: report_path is {report_path}, type: {type(report_path)}")

# 要刪除的列
columns_to_drop = ['Environment', 'Cloud Provider', 'Cloud Provider Identifier', 'Cost', 'Savings', 'Provider Resource ID', 'Tags', 'Check ID', 'Link to resource', 'Meta', 'TagObjects']

# 讀取 ECV_Azure_Recommendations.csv
recommendations_df = pd.read_csv(recommendations_path)
recommendations_dict = dict(zip(recommendations_df['Rule ID'], recommendations_df['Suggestion']))

# 處理每個CSV文件
for csv_file in os.listdir(cspm_raw_data_path):
    if csv_file.endswith('.csv'):
        print(f"正在處理文件: {csv_file}")
        
        # 讀取CSV文件
        df = pd.read_csv(cspm_raw_data_path / csv_file)
        
        # 刪除不需要的列
        df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
        
        # 添加新的列：ECV Suggestion 和 Command
        df['ECV Suggestion'] = df['Rule ID'].map(recommendations_dict)
        df['Command'] = ''
        
        # 確保 'Account Name' 列存在
        if 'Account Name' not in df.columns:
            print("警告: 'Account Name' 列不存在")
            continue
        
        # 分離最後兩個Account Name
        df_main = df.iloc[:-2]
        df_last_two = df.tail(2)
        
        # 保存主要數據CSV文件
        main_file_name = f"processed_main_{csv_file}"
        main_file_path = report_path / main_file_name
        df_main.to_csv(main_file_path, index=False)
        print(f"已生成主要數據文件: {main_file_path}")
        
        # 保存最後兩個賬戶的CSV文件
        last_two_file_name = f"processed_last_two_{csv_file}"
        last_two_file_path = report_path / last_two_file_name
        df_last_two.to_csv(last_two_file_path, index=False)
        print(f"已生成最後兩個賬戶文件: {last_two_file_path}")
        
        print("-----------------------------------")

print("所有文件處理完成。")

import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, PageBreak, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import io
from pathlib import Path
import matplotlib.font_manager as fm

# 註冊字型
font_path = "/home/leo/app/fonts/MSJH.TTC"
pdfmetrics.registerFont(TTFont("MSJH", font_path))

# 設置 matplotlib 使用的字型
plt.rcParams['font.sans-serif'] = ['MSJH']
plt.rcParams['axes.unicode_minus'] = False

# 創建字體屬性對象
font_prop = fm.FontProperties(fname=font_path)

def read_csv_file(file_path):
    print(f"嘗試讀取文件: {file_path}")
    print(f"文件是否存在: {file_path.exists()}")
    
    if not file_path.exists():
        raise FileNotFoundError(f"文件 {file_path} 不存在")
    
    df = pd.read_csv(file_path)
    return df

def create_charts(df):
    charts = []
    
    # 1. Risk Level 分布（排除 NOT SCORED 和 SUCCESS）
    df_risk = df[(df['Check Status'] != 'NOT SCORED') & (df['Check Status'] != 'SUCCESS')]
    risk_dist = df_risk['Risk Level'].value_counts()
    plt.figure(figsize=(10, 6))
    plt.pie(risk_dist, labels=risk_dist.index, autopct='%1.1f%%', startangle=90)
    plt.title('風險等級分佈\n(不包括 NOT SCORED 和 SUCCESS)', fontproperties=font_prop, fontsize=14)
    plt.axis('equal')
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('風險等級分佈', img_buffer))
    plt.close()

    # 2. Top 5 Rule ID
    rule_id_counts = df['Rule ID'].value_counts().nlargest(5)
    plt.figure(figsize=(10, 6))
    rule_id_counts.plot(kind='barh')
    plt.title('前 5 個最常見的規則 ID', fontproperties=font_prop, fontsize=14)
    plt.xlabel('數量', fontproperties=font_prop)
    plt.ylabel('規則 ID', fontproperties=font_prop)
    plt.tight_layout()
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('前 5 個最常見的規則 ID', img_buffer))
    plt.close()

    # 3. Service 分布
    service_dist = df['Service'].value_counts()
    plt.figure(figsize=(10, 6))
    service_dist.plot(kind='bar')
    plt.title('服務分佈', fontproperties=font_prop, fontsize=14)
    plt.xlabel('服務', fontproperties=font_prop)
    plt.ylabel('數量', fontproperties=font_prop)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('服務分佈', img_buffer))
    plt.close()

    # 4. Categories 分布
    categories = df['Categories'].str.split(', ', expand=True).stack().value_counts()
    plt.figure(figsize=(12, 6))
    categories.plot(kind='bar')
    plt.title('類別分佈', fontproperties=font_prop, fontsize=14)
    plt.xlabel('類別', fontproperties=font_prop)
    plt.ylabel('數量', fontproperties=font_prop)
    plt.xticks(rotation=0, ha='center')
    plt.tight_layout()
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('類別分佈', img_buffer))
    plt.close()

    # 5. Check Status 分布
    status_dist = df['Check Status'].value_counts()
    plt.figure(figsize=(10, 6))
    plt.pie(status_dist, labels=status_dist.index, autopct='%1.1f%%', startangle=90)
    plt.title('檢查狀態分佈', fontproperties=font_prop, fontsize=14)
    plt.axis('equal')
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('檢查狀態分佈', img_buffer))
    plt.close()

    return charts

def generate_pdf_report(charts, output_path):
    output_path_str = str(output_path)
    
    doc = SimpleDocTemplate(output_path_str, pagesize=A4)
    elements = []
    styles = getSampleStyleSheet()

    # 創建使用自定義字型的樣式
    title_style = ParagraphStyle('CustomTitle', parent=styles['Title'], fontName='MSJH')
    heading_style = ParagraphStyle('CustomHeading2', parent=styles['Heading2'], fontName='MSJH')

    elements.append(Paragraph("CSPM 分析報告", title_style))
    elements.append(Paragraph("", styles['Normal']))  # 添加一些空間
    
    for title, img_buffer in charts:
        elements.append(Paragraph(title, heading_style))
        img = Image(img_buffer, width=6.5*inch, height=4*inch)
        elements.append(img)
        elements.append(PageBreak())

    doc.build(elements)

def main():
    file_path = Path(f'{main_file_path}')
    output_path = file_path.parent / 'cspm_analysis_report.pdf'

    try:
        df = read_csv_file(file_path)
        charts = create_charts(df)
        generate_pdf_report(charts, output_path)
        print(f"PDF報告已生成: {output_path}")
    except FileNotFoundError as e:
        print(f"錯誤：無法找到CSV文件。{e}")
    except Exception as e:
        print(f"生成報告時發生錯誤：{e}")
        print(f"錯誤類型: {type(e)}")
        print(f"錯誤詳情: {str(e)}")

if __name__ == "__main__":
    main()

In [106]:
import os
import io
import getpass
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pathlib import Path
from reportlab.lib.units import inch, cm
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, PageBreak, Image, Table, TableStyle
from reportlab.lib import colors
from reportlab.platypus.tableofcontents import TableOfContents
from reportlab.platypus.doctemplate import PageTemplate, BaseDocTemplate
from reportlab.platypus.frames import Frame

# 獲取當前日期並格式化成 'YYYYMM' 的形式 #202407
currentmonth_date = datetime.datetime.now().strftime('%Y%m')
# 獲取當前月份 #7
current_month = datetime.datetime.now().month
# 取得當天的日期 
today = datetime.datetime.now().strftime('%Y%m%d')
# 報告月份與位置
report_file_name = f"CSPM_{currentmonth_date}_Report"
ecv_recommendations = 'ECV_Azure_Recommendations.csv'
# 匯出檔案路徑
username = getpass.getuser() #leo
local_path = Path(rf"/home/{username}/app")
recommendations_path = Path(f'{local_path}/Recommendations/{ecv_recommendations}')
cspm_raw_data_path = Path(rf'{local_path}/cspm_raw_data')
report_path = Path(f'{local_path}/report')

print(f"Debug: report_path is {report_path}, type: {type(report_path)}")

# 要刪除的列
columns_to_drop = [
    'Environment', 'Cloud Provider', 'Cloud Provider Identifier',
    'Cost', 'Savings', 'Provider Resource ID', 'Tags', 'Check ID',
    'Link to resource', 'Meta', 'TagObjects'
]

# 註冊字型
font_path = "/home/leo/app/fonts/MSJH.TTC"
pdfmetrics.registerFont(TTFont("MSJH", font_path))

# 設置 matplotlib 使用的字型
plt.rcParams['font.sans-serif'] = ['MSJH']
plt.rcParams['axes.unicode_minus'] = False

# 創建字體屬性對象
font_prop = fm.FontProperties(fname=font_path)

def create_cover(title, date):
    """創建封面"""
    cover_style = ParagraphStyle(
        'Cover',
        fontName='MSJH',
        fontSize=24,
        leading=30,
        alignment=1,  # 居中
    )
    date_style = ParagraphStyle(
        'Date',
        fontName='MSJH',
        fontSize=12,
        leading=14,
        alignment=1,  # 居中
    )
    elements = [
        Paragraph(title, cover_style),
        Paragraph("<br/><br/><br/>", cover_style),  # 添加一些空間
        Paragraph(date, date_style),
        PageBreak()
    ]
    return elements

def create_toc():
    """創建目錄"""
    toc = TableOfContents()
    toc.levelStyles = [
        ParagraphStyle(name='TOCHeading1', fontSize=14, fontName='MSJH'),
        ParagraphStyle(name='TOCHeading2', fontSize=12, fontName='MSJH', leftIndent=20),
    ]
    return [Paragraph("目錄", ParagraphStyle(name='Heading1', fontSize=16, fontName='MSJH')), toc, PageBreak()]

def process_csv_file(csv_file_path, recommendations_dict):
    df = pd.read_csv(csv_file_path)
    
    # 刪除不需要的列
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    
    # 添加新的列：ECV Suggestion 和 Command
    df['ECV Suggestion'] = df['Rule ID'].map(recommendations_dict)
    df['Command'] = ''
    
    # 過濾出 'Check Status' 為 'FAILURE' 的數據
    df_failure = df[df['Check Status'] == 'FAILURE']
    
    return df_failure, df

def create_charts(df):
    charts = []
    
    # 1. Risk Level 分布（排除 NOT SCORED 和 SUCCESS）
    df_risk = df[(df['Check Status'] != 'NOT SCORED') & (df['Check Status'] != 'SUCCESS')]
    risk_dist = df_risk['Risk Level'].value_counts()
    plt.figure(figsize=(8, 4.5))
    plt.pie(risk_dist, labels=risk_dist.index, autopct='%1.1f%%', startangle=90)
    plt.title('風險等級分佈\n(不包括 NOT SCORED 和 SUCCESS)', fontproperties=font_prop, fontsize=14)
    plt.axis('equal')
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('風險等級分佈', img_buffer))
    plt.close()

    # 2. Top 5 Rule ID
    rule_id_counts = df['Rule ID'].value_counts().nlargest(5)
    plt.figure(figsize=(8, 4.5))
    rule_id_counts.plot(kind='barh')
    plt.title('前 5 個最常見的規則 ID', fontproperties=font_prop, fontsize=14)
    plt.xlabel('數量', fontproperties=font_prop)
    plt.ylabel('規則 ID', fontproperties=font_prop)
    plt.tight_layout()
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('前 5 個最常見的規則 ID', img_buffer))
    plt.close()

    # 3. Service 分布
    service_dist = df['Service'].value_counts()
    plt.figure(figsize=(8, 4.5))
    service_dist.plot(kind='bar')
    plt.title('服務分佈', fontproperties=font_prop, fontsize=14)
    plt.xlabel('服務', fontproperties=font_prop)
    plt.ylabel('數量', fontproperties=font_prop)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('服務分佈', img_buffer))
    plt.close()

    # 4. Categories 分布
    categories = df['Categories'].str.split(', ', expand=True).stack().value_counts()
    plt.figure(figsize=(8, 4.5))
    categories.plot(kind='bar')
    plt.title('類別分佈', fontproperties=font_prop, fontsize=14)
    plt.xlabel('類別', fontproperties=font_prop)
    plt.ylabel('數量', fontproperties=font_prop)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('類別分佈', img_buffer))
    plt.close()

    # 5. Check Status 分布
    status_dist = df['Check Status'].value_counts()
    plt.figure(figsize=(8, 4.5))
    plt.pie(status_dist, labels=status_dist.index, autopct='%1.1f%%', startangle=90)
    plt.title('檢查狀態分佈', fontproperties=font_prop, fontsize=14)
    plt.axis('equal')
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png')
    img_buffer.seek(0)
    charts.append(('檢查狀態分佈', img_buffer))
    plt.close()

    return charts

def create_recommendation_table(df):
    styles = getSampleStyleSheet()
    style = ParagraphStyle(
        'TableContent',
        parent=styles['BodyText'],
        fontName='MSJH',
        fontSize=7,
        leading=9,
        wordWrap='CJK'
    )

    # 創建一個函數來生成段落
    def create_paragraph(text):
        return Paragraph(str(text), style)

    # 準備表格數據
    data = [['Rule ID', 'Rule Title', 'Risk Level', 'ECV Suggestion']]
    for _, row in df.iterrows():
        data.append([
            create_paragraph(row['Rule ID']),
            create_paragraph(row['Rule Title']),
            create_paragraph(row['Risk Level']),
            create_paragraph(row['ECV Suggestion'])
        ])
    
    # 創建表格
    table = Table(data, colWidths=[2*cm, 5*cm, 2*cm, 10*cm])
    
    # 設置表格樣式
    table_style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
        ('FONTNAME', (0, 0), (-1, -1), 'MSJH'),  # 所有單元格使用微軟正黑體
        ('FONTSIZE', (0, 0), (-1, 0), 8),  # 表頭字體大小
        ('BOTTOMPADDING', (0, 0), (-1, 0), 6),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
        ('VALIGN', (0, 0), (-1, -1), 'TOP'),
        ('GRID', (0, 0), (-1, -1), 0.5, colors.black)
    ])
    table.setStyle(table_style)
    
    return table

class CSPMReport(BaseDocTemplate):
    def __init__(self, filename, **kw):
        super().__init__(filename, **kw)
        page_width, page_height = A4
        frame = Frame(
            inch, inch, 
            page_width - 2*inch, 
            page_height - 2*inch, 
            id='normal'
        )
        template = PageTemplate(id='normal', frames=frame)
        self.addPageTemplates([template])

    def afterFlowable(self, flowable):
        """Registers TOC entries."""
        if flowable.__class__.__name__ == 'Paragraph':
            text = flowable.getPlainText()
            style = flowable.style.name
            if style == 'Heading1':
                self.notify('TOCEntry', (0, text, self.page))
            if style == 'Heading2':
                self.notify('TOCEntry', (1, text, self.page))

def generate_pdf_report(charts, recommendation_table, output_path):
    output_path_str = str(output_path)
    
    doc = CSPMReport(output_path_str, pagesize=A4)
    
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle('Title', parent=styles['Title'], fontName='MSJH', fontSize=16)
    heading1_style = ParagraphStyle('Heading1', parent=styles['Heading1'], fontName='MSJH', fontSize=14)
    heading2_style = ParagraphStyle('Heading2', parent=styles['Heading2'], fontName='MSJH', fontSize=12)
    
    elements = []
    
    # 添加封面
    elements.extend(create_cover("CTBC CSPM Monthly Report", datetime.date.today().strftime("%Y-%m-%d")))
    
    # 添加目錄
    elements.extend(create_toc())
    
    # 添加圖表
    elements.append(Paragraph("分析圖表", heading1_style))
    for title, img_buffer in charts:
        elements.append(Paragraph(title, heading2_style))
        img = Image(img_buffer, width=6*inch, height=3.5*inch)
        elements.append(img)
        elements.append(PageBreak())
    
    # 添加建議表格
    elements.append(Paragraph("失敗項目及建議", heading1_style))
    elements.append(recommendation_table)
    
    doc.multiBuild(elements)

def main():
    # 讀取 ECV_Azure_Recommendations.csv
    recommendations_df = pd.read_csv(recommendations_path)
    recommendations_dict = dict(zip(recommendations_df['Rule ID'], recommendations_df['Suggestion']))

    # 處理每個CSV文件
    for csv_file in os.listdir(cspm_raw_data_path):
        if csv_file.endswith('.csv'):
            print(f"正在處理文件: {csv_file}")
            
            csv_file_path = cspm_raw_data_path / csv_file
            df_failure, df = process_csv_file(csv_file_path, recommendations_dict)
            
            # 生成圖表
            charts = create_charts(df)
            
            # 創建建議表格
            recommendation_table = create_recommendation_table(df_failure)
            
            # 生成PDF報告
            output_file_name = f"{csv_file[:-4]}.pdf"
            output_path = report_path / output_file_name
            
            try:
                generate_pdf_report(charts, recommendation_table, output_path)
                print(f"PDF報告已生成: {output_path}")
            except Exception as e:
                print(f"生成報告時發生錯誤：{e}")
                print(f"錯誤類型: {type(e)}")
                print(f"錯誤詳情: {str(e)}")
            
            print("-----------------------------------")

    print("所有文件處理完成。")

if __name__ == "__main__":
    main()

Debug: report_path is /home/leo/app/report, type: <class 'pathlib.PosixPath'>
正在處理文件: ECV-CTBCMSSP-Connectivity-MonthlyReport-202406.csv


findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of 

PDF報告已生成: /home/leo/app/report/ECV-CTBCMSSP-Connectivity-MonthlyReport-202406.pdf
-----------------------------------
正在處理文件: ECV-CTBCMSSP-Landing Zone (ARO)-MonthlyReport-202406.csv


findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of the following families were found: MSJH
findfont: Generic family 'sans-serif' not found because none of 

PDF報告已生成: /home/leo/app/report/ECV-CTBCMSSP-Landing Zone (ARO)-MonthlyReport-202406.pdf
-----------------------------------
所有文件處理完成。
